In [58]:
import os
os.chdir('../')

In [59]:
print("현재 디렉토리 " , os.getcwd())

현재 디렉토리  /Users/mac/Desktop


In [60]:
import time

from TS import (
    UpbitTrader,
    UpbitDataProvider,
    StrategyBuyAndHold,
    Operator,
    LogManager,
    Analyzer,
)

In [61]:
operator = Operator()

operator.initialize(
    UpbitDataProvider(),
    StrategyBuyAndHold(),
    UpbitTrader(),
    Analyzer(),
    budget=29975.73614204)

create output folder


2022-09-28 13:49:25,310 DEBUG   UpbitTrader-Worker - Worker[UpbitTrader-Worker: 123145881755648] WAIT =========


### 체결 정보

In [69]:
import jwt
import hashlib
import os
import requests
import uuid
from urllib.parse import urlencode, unquote

access_key = os.environ['UPBIT_OPEN_API_ACCESS_KEY']
secret_key = os.environ['UPBIT_OPEN_API_SECRET_KEY']
server_url = os.environ['UPBIT_OPEN_API_SERVER_URL']

params = {'uuid': '508d57ee-c08b-4c4f-bec3-cbe1e188b998'}
query_string = unquote(urlencode(params, doseq=True)).encode("utf-8")

m = hashlib.sha512()
m.update(query_string)
query_hash = m.hexdigest()

payload = {
    'access_key': operator.trader.ACCESS_KEY,
    'nonce': str(uuid.uuid4()),
    'query_hash': query_hash,
    'query_hash_alg': 'SHA512'}

jwt_token = jwt.encode(payload, operator.trader.SECRET_KEY)
authorization = 'Bearer {}'.format(jwt_token)
headers = {'Authorization': authorization}

res = requests.get(server_url + '/v1/order', params=params, headers=headers)
res.json()

{'uuid': '508d57ee-c08b-4c4f-bec3-cbe1e188b998',
 'side': 'ask',
 'ord_type': 'market',
 'state': 'done',
 'market': 'KRW-BTC',
 'created_at': '2022-09-28T14:51:58+09:00',
 'volume': '0.000367',
 'remaining_volume': '0',
 'reserved_fee': '0',
 'remaining_fee': '0',
 'paid_fee': '4.990833',
 'locked': '0',
 'executed_volume': '0.000367',
 'trades_count': 1,
 'trades': [{'market': 'KRW-BTC',
   'uuid': '987d734c-a0d8-4043-b34f-55d36ba96bfb',
   'price': '27198000',
   'volume': '0.000367',
   'funds': '9981.666',
   'trend': 'down',
   'created_at': '2022-09-28T14:51:57+09:00',
   'side': 'ask'}]}

### 주문 가능 금액

In [12]:
import uuid
import jwt
import requests

payload = {'access_key': operator.trader.ACCESS_KEY, 'nonce': str(uuid.uuid4())}
jwt_token = jwt.encode(payload, operator.trader.SECRET_KEY)
authorization = 'Bearer {}'.format(jwt_token)
headers = {'Authorization': authorization}

res = requests.get(operator.trader.SERVER_URL + '/v1/accounts', headers=headers)
res.json()

[{'currency': 'KRW',
  'balance': '29975.73614204',
  'locked': '0',
  'avg_buy_price': '0',
  'avg_buy_price_modified': True,
  'unit_currency': 'KRW'}]

In [34]:
operator.analyzer.make_start_point()

2022-09-25 21:02:05,845 DEBUG          UpbitTrader - account info | banance: 29975.73614204 | {'BTC': (0, 0)} | {'BTC': 27410000.0}
2022-09-25 21:02:05,846 DEBUG             Analyzer - make score record new_info {'balance': 29975.73614204, 'asset': {'BTC': (0, 0)}, 'quote': {'BTC': 27410000.0}, 'date_time': '2022-09-25T21:02:05'}
2022-09-25 21:02:05,847 DEBUG             Analyzer - BTC| yield record: 0 | buy avg: 0.0 | price: 27410000.0 | amount: 0.0
2022-09-25 21:02:05,849 DEBUG             Analyzer - price change ratio 27410000.0 -> 27410000.0, 0
2022-09-25 21:02:05,850  INFO             Analyzer - cumulative return 29976 -> 29975.73614204, -0.001


In [35]:
import threading

thread = threading.Thread(target=operator._execute_trading)
thread.start()
thread.join()

2022-09-25 21:02:07,080 DEBUG             Operator - ========= trading is started =========


끝이당


In [6]:
# call back

def send_request_callback(result):
    print(result)

In [ ]:
"""
한 번의 Execute trading 과정
"""
import hashlib
import uuid
import jwt

operator.analyzer.make_start_point()

# 종목 데이터 업데이트
trading_info = operator.data_provider.get_info()
operator.strategy.update_trading_info(trading_info)
operator.analyzer.put_trading_info(trading_info)

# 주문 생성
target_request = operator.strategy.get_request()

# 체결 결과 생성
operator.trader.send_request(target_request, send_request_callback)

print(" ========= trading info ========= ")
print(trading_info)
print(" ========= request info ========= ")
print(target_request)
print(" ========= result info ========= ")

In [9]:
operator.strategy.data

[{'market': 'KRW-BTC',
  'date_time': '2022-09-25T11:52:00',
  'opening_price': 27269000.0,
  'high_price': 27277000.0,
  'low_price': 27264000.0,
  'closing_price': 27264000.0,
  'acc_price': 32996676.46198,
  'acc_volume': 1.21017616}]

In [55]:
operator.trader.order_map

{'1664074365.563': {'uuid': '217ffe1b-9f54-4a03-9e13-62746f4d1be4',
  'callback': <function __main__.send_request_callback(result)>,
  'result': {'uuid': '217ffe1b-9f54-4a03-9e13-62746f4d1be4',
   'state': 'requested',
   'request': {'id': '1664074365.563',
    'type': 'buy',
    'price': 5997.853454152,
    'amount': 0.0002,
    'date_time': '2022-09-25T11:52:45'},
   'type': 'buy',
   'price': 5997.853454152,
   'amount': 0.0002,
   'msg': 'success'}}}